In [48]:
import pandas as pd
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

# export management

In [2]:
# upload export file
ref_ga_path = "../../scripts/export google analytics/update/ga_referrer.csv"
#new_ref_df = pd.read_csv(ref_ga_path)
#save_dataframe_to_file("qgiv-stats-data", "ga_referrer.csv", pd.read_csv(ref_ga_path))

In [50]:
# merging exports
#ga_referrers = ga_referrers.append(pd.read_csv(ref_ga_path))
#ga_referrers.head(3)

5902353


/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


8199066


In [4]:
ga_referrers = get_dataframe_from_file("qgiv-stats-data", "ga_referrer.csv")

In [17]:
original_len = len(ga_referrers)
ga_referrers.drop_duplicates(inplace=True)
ga_referrers['date'].min(), ga_referrers['date'].max(), len(ga_referrers), original_len

('2020-03-16', '2020-11-07', 4888602, 4888602)

In [4]:
ga_referrers['qgiv_frontend'] = ga_referrers['path'].str.startswith('/for/')
ga_referrers['p2p_frontend'] = ga_referrers['path'].str.startswith('/event/')

In [29]:
ga_referrers['path'] = ga_referrers['path'].str[0:250].str.replace(',', '')
ga_referrers['source'] = ga_referrers['source'].str[0:250].str.replace(',', '')

cols = ['date', 'org', 'form', 'views', 'path', 'source', 'qgiv_frontend', 'p2p_frontend']
save_dataframe_to_file("qgiv-stats-data", "ga_referrer.csv", ga_referrers[cols])

uploading to S3
Done


In [30]:
q = '''copy googleanalytics_referrer
        from 's3://qgiv-stats-data/ga_referrer.csv'
        iam_role 'arn:aws:iam::637885584661:role/AWSRoleForRedshift'
        emptyasnull
        blanksasnull
        fillrecord
        delimiter ','
        ignoreheader 1
        region 'us-east-1';'''
redshift_query_write(q, schema='public')

### tagging based on /for/ and /event/

In [5]:
ga_traffic = ga_referrers[~ga_referrers['qgiv_frontend']&~ga_referrers['p2p_frontend']].copy()
ga_traffic['org'] = 0
ga_traffic['form'] = 0
frontend_ga_traffic = ga_referrers[ga_referrers['qgiv_frontend']|ga_referrers['p2p_frontend']].copy()

forms = redshift_query_read("select form, org, path from forms")
for _, r in forms.iterrows():
    # tag org & form
    p_q = "/for/{}".format(r['path'])
    p_e = "/event/{}".format(r['path'])
    these_ga = frontend_ga_traffic[(frontend_ga_traffic['path'].str.startswith(p_q))|(frontend_ga_traffic['path'].str.startswith(p_e))].copy()
    these_ga['org'] = r['org']
    these_ga['form'] = r['form']
    
    ga_traffic = ga_traffic.append(these_ga)

### tagging based on formId= or org= in path

In [45]:
ga_referrers = redshift_query_read("select * from googleanalytics_referrer")

In [46]:
ga_referrers.tail(3)

,date,org,form,views,path,source,qgiv_frontend,p2p_frontend
4201554,2020-02-08,NaN,952721,1,/account/create/embed/?p2pRegistration=1&formI...,(direct),False,False
4201555,2020-02-08,NaN,97335,4,/account/login/embed/?formId=97335,(direct),False,False
4201556,2020-02-08,NaN,946857,1,/account/login/embed/54268/?formId=946857,horizonhonors.org,False,False


----
----
testing

In [92]:
na_orgs = ga_referrers[ga_referrers['org'].isna()].copy()
na_orgs['form'] = na_orgs['form'].astype(int)
forms = redshift_query_read("select id as form, org, path from form", schema='production')
forms['form'] = forms['form'].astype(int)

In [93]:
print(len(na_orgs[na_orgs['org'].isna()]))
na_orgs['org'] = na_orgs['form'].apply(lambda x: forms[forms['form']==x]['org'].iloc[0] if len(forms[forms['form']==x]) > 0 else 0)
print(len(na_orgs[na_orgs['org']==0]))

206391
8


In [97]:
na_orgs[na_orgs['org']==0]['path'].iloc[0]

'/account/login/embed/?transaction=43984556&email=ybeel_b@ubgzbvy.dbz&formId=283520'

In [99]:
len_na = len(ga_referrers[ga_referrers['org'].isna()])
len_zero = len(ga_referrers[ga_referrers['org']==0])
print("NA: {}; 0: {}".format(len_na, len_zero))

ga_referrers = ga_referrers.append(na_orgs).drop_duplicates(subset=['date', 'views', 'path', 'source'], keep='last')

len_na = len(ga_referrers[ga_referrers['org'].isna()])
len_zero = len(ga_referrers[ga_referrers['org']==0])
print("NA: {}; 0: {}".format(len_na, len_zero))

NA: 206391; 0: 268192
NA: 0; 0: 268200


---
---

In [21]:
msk_path = (ga_referrers['path'].str.contains('formId='))
no_org = ga_referrers[(ga_referrers['org']==0)&msk_path]
extracted_forms = no_org['path'].str.extract('formId=(?P<form>[0-9]+)').dropna()['form'].unique().tolist()

print("{} entries w/ 0 org; {} extracted form IDs to tag".format(len(ga_referrers[ga_referrers['org']==0]), len(extracted_forms)))

msk_path = (ga_traffic['path'].str.contains('formId='))
no_org = ga_traffic[(ga_traffic['org']==0)&msk_path].copy()

no_org['form'] = no_org['path'].str.extract('formId=(?P<form>[0-9]+)')['form']
no_org['org'] = no_org['path'].str.extract('org=(?P<org>[0-9]+)')['org']

ga_traffic = ga_traffic.append(no_org).drop_duplicates(subset=['date', 'views', 'path', 'source'], keep='last')
    
ga_referrers = ga_referrers.drop_duplicates(subset=['date', 'views', 'path', 'source'], keep='last')
print("{} entries w/ 0 org".format(len(ga_referrers[ga_referrers['org']==0])))

432899 entries w/ 0 org; 2460 extracted form IDs to tag
432899 entries w/ 0 org


In [35]:
ga_referrers['date'].min(), ga_referrers['date'].max()

(Timestamp('2019-12-29 00:00:00'), Timestamp('2020-11-07 00:00:00'))

In [8]:
cols = ['date', 'org', 'form', 'views', 'path', 'source', 
        'qgiv_frontend', 'p2p_frontend', 'facebook', 'twitter']
#save_dataframe_to_file("qgiv-stats-data", "ga_referrer.csv", ga_referrers[cols])

uploading to S3
Done


In [103]:
ga_referrers['org'] = ga_referrers['org'].astype(int)

# review

In [49]:
#ga_referrers = get_dataframe_from_file("qgiv-stats-data", "ga_referrer.csv")
q = "select * from googleanalytics_referrer"
ga_referrers = redshift_query_read(q, schema="public")

In [50]:
len(ga_referrers[ga_referrers['org']==0]), len(ga_referrers[ga_referrers['org'].isna()])

(268200, 0)

In [51]:
len_noorg = len(ga_referrers[ga_referrers['org']==0])
len_noform = len(ga_referrers[ga_referrers['form']==0])
len_fb = len(ga_referrers[ga_referrers['facebook']])
perc_fb = (len_fb / len(ga_referrers)) * 100.
len_tw = len(ga_referrers[ga_referrers['twitter']])
perc_tw = (len_tw / len(ga_referrers)) * 100.

soc_msk = ga_referrers['facebook']|ga_referrers['twitter']
org_msk = ga_referrers['org']!=0
len_soc_org = len(ga_referrers[(soc_msk)&(org_msk)])
perc_soc_org = (len_soc_org / len(ga_referrers)) * 100.
count_orgs_soc = len(ga_referrers[(soc_msk)&(org_msk)]['org'].unique())
count_forms_soc = len(ga_referrers[(soc_msk)&(org_msk)]['form'].unique())

print("{} ({:.2f}%) no org".format(len_noorg, (len_noorg / len(ga_referrers)) * 100.))
print("{} ({:.2f}%) no form".format(len_noform, (len_noform / len(ga_referrers)) * 100.))
print("{} ({:.2f}%) facebook; {} ({:.2f}%) twitter".format(len_fb, perc_fb, len_tw, perc_tw))
print("{} ({:.2f}%) social and has org; {} unique orgs; {} unique forms".format(len_soc_org, perc_soc_org, count_orgs_soc, count_forms_soc))

268200 (6.38%) no org
268192 (6.38%) no form
764186 (18.19%) facebook; 19482 (0.46%) twitter
775941 (18.47%) social and has org; 1338 unique orgs; 2759 unique forms


In [43]:
ga_referrers['is_social'] = soc_msk
org_traff = []
for o in ga_referrers['org'].unique().tolist():
    this_traffic = ga_referrers[ga_referrers['org']==o]
    org_traff.append({
        'org': o,
        'social_traffic': this_traffic[this_traffic['is_social']]['views'].sum(),
        'nonsocial_traffic': this_traffic[~this_traffic['is_social']]['views'].sum()
    })
org_traff = pd.DataFrame(org_traff)[['org', 'social_traffic', 'nonsocial_traffic']]
org_traff['all_views'] = org_traff['social_traffic'] + org_traff['nonsocial_traffic']
org_traff['soc_perc'] = org_traff['social_traffic'] / org_traff['all_views']

In [53]:
soc_msk = org_traff['social_traffic']>0
mn_social_views_act = org_traff[soc_msk]['social_traffic'].mean()
mn_social_views_all = org_traff['social_traffic'].mean()
mn_social_views_perc_act = org_traff[soc_msk]['soc_perc'].mean()
mn_social_views_perc_all = org_traff['soc_perc'].mean()

In [54]:
len_orgs = len(org_traff[soc_msk])
len_all_orgs = len(ga_referrers['org'].unique())

print("{} ({:.2f}%) orgs with social traffic".format(len_orgs, (len_orgs / len_all_orgs) * 100.))
print("{:.2f} average social views for socially active orgs".format(mn_social_views_act))
print("{:.2f} average social views for all orgs".format(mn_social_views_all))
print("{:.2f}% of per org traffic is from social sources for socially active orgs".format(mn_social_views_perc_act))
print("{:.2f}% of per org traffic is from social sources for all orgs".format(mn_social_views_perc_all))

1339 (41.15%) orgs with social traffic
1276.42 average social views for socially active orgs
525.24 average social views for all orgs
0.12% of per org traffic is from social sources for socially active orgs
0.05% of per org traffic is from social sources for all orgs


In [67]:
ranges = [(0, 25), (25, 50), (50, 100), (100, 500), (500, 2500),
          (2500, 5000), (5000)]
print("Orgs by social view counts:")
for r in ranges:
    try:
        len_orgs = len(org_traff[(org_traff['social_traffic']>=r[0])&(org_traff['social_traffic']<r[1])])
    except:
        len_orgs = len(org_traff[org_traff['social_traffic']>=r])
    print("{}: {}".format(r, len_orgs))

Orgs by social view counts:
(0, 25): 2528
(25, 50): 143
(50, 100): 128
(100, 500): 226
(500, 2500): 151
(2500, 5000): 36
5000: 42


In [65]:
ranges = [(0, 5), (5, 10), (10, 15), (15, 25), (25, 50), (50, 75), 
          (75, 85), (85, 95), (95)]
print("Orgs by social view percentages:")
for r in ranges:
    try:
        len_orgs = len(org_traff[(org_traff['social_traffic']>=r[0])&(org_traff['social_traffic']<r[1])])
    except:
        len_orgs = len(org_traff[org_traff['social_traffic']>=r])
    print("{}: {}".format(r, len_orgs))

Orgs by social view percentages:
(0, 5): 2207
(5, 10): 125
(10, 15): 94
(15, 25): 102
(25, 50): 143
(50, 75): 81
(75, 85): 18
(85, 95): 22
95: 462
